# Tweem.io Similarity Comparison

### Imports

In [1]:
#  Necessary to allow assembly framework to run
import os
os.chdir('..')

In [2]:
import pandas as pd
import numpy as np
import inspect
import importlib

import assembly
from assembly import config as asmbl_config
from assembly import db as asmbl_db
from assembly import models as asmbl_models

from lib.tweemio import twm
from lib.tweemio import similarity
from lib.tweemio import twitter

In [3]:
# importlib.reload(similarity)
# importlib.reload(filepersist)

### Settings

In [3]:
os.environ['ASSEMBLY_ENV'] = 'Development'
app = assembly.Assembly.init(__name__, {'default': []})

### Calculate Similarity to Group

In [4]:
results = twm.calculate(app.config, screen_name='barackobama', group='trumpian', force=True) #, compare_to_screen_names=['realdonaldtrump'])

In [44]:
{screen_name : np.mean(result.y_prob) for screen_name, result in results.items()}

{'realdonaldtrump': 0.0362871456636854,
 'kellyannepolls': 0.05737222991201928,
 'secpompeo': 0.11167948068663866,
 'mike_pence': 0.06720481314569705,
 'devinnunes': 1.9921557356344103e-05,
 'lindseygrahamsc': 0.007312821960141586,
 'donaldjtrumpjr': 0.0015974980293346611,
 'rudygiuliani': 0.0007456474545912521,
 'govmikehuckabee': 0.00879048543646449,
 'alandersh': 0.00044566331079554106,
 'repmattgaetz': 0.0010570736682972686,
 'jim_jordan': 0.0046734620896466015,
 'repmarkmeadows': 0.0002013814121971472,
 'gopleader': 0.008936633656353447,
 'gtconway3d': 0.00020415448715250007,
 'scaramucci': 0.01527626348948547}

### Scratchpad

In [68]:
screen_name='barackobama'

In [66]:
tapi = twitter.TwitterApi(app.config['TWITTER_API_CREDS'])
tweets = tapi.timeline(screen_name, condense_factor=app.config['TWEET_CONDENSE_FACTOR'])

In [74]:
nlp_parser = similarity.get_nlp_readability()

nlpdocs = [nlp_parser(tweet) for tweet in tweets]

fkgl  = [nlpdoc._.flesch_kincaid_grade_level for nlpdoc in nlpdocs]
fkre  = [nlpdoc._.flesch_kincaid_reading_ease for nlpdoc in nlpdocs]
dc    = [nlpdoc._.dale_chall for nlpdoc in nlpdocs]
cl    = [nlpdoc._.coleman_liau_index for nlpdoc in nlpdocs]
autor = [nlpdoc._.automated_readability_index for nlpdoc in nlpdocs]

{
    'flesch_kincaid_grade_level': np.mean(fkgl),
    'flesch_kincaid_reading_ease': np.mean(fkre),
    'dale_chall': np.mean(dc),
    'coleman_liau_index': np.mean(cl),
    'automated_readability_index': np.mean(autor)
}

In [ ]:
# ============================================================================================

In [ ]:
# POC w/ google cloud service
# Load creds locally, then set into credentials object

In [131]:
import json
from google.cloud import storage
from google.oauth2 import service_account

In [148]:
with open('/Users/liangjh/workspace/tspire-api/app/resources/gcs-tspire-creds.json', 'r') as f:
    credinfo = json.load(f)

credentials = service_account.Credentials.from_service_account_info(credinfo)
storage_client = storage.Client('tspire', credentials=credentials)

In [153]:
bucket = storage_client.get_bucket('images.trumpspired.com')

In [155]:
list(bucket.list_blobs())

[<Blob: images.trumpspired.com, generated/, 1567609269993804>,
 <Blob: images.trumpspired.com, generated/002fd842e6f74ddebdd88895bdf2b499.jpg, 1568402588096749>,
 <Blob: images.trumpspired.com, generated/015c98aea154459da4368292b7246856.jpg, 1572234470175203>,
 <Blob: images.trumpspired.com, generated/021bee11ce50497d92d03aedee786a33.jpg, 1571255537262017>,
 <Blob: images.trumpspired.com, generated/0228a64b364741d9ae39febbe787e2e3.jpg, 1567609267886457>,
 <Blob: images.trumpspired.com, generated/0231fb4f84574afa9b1f73673ad3eca1.jpg, 1569707713524534>,
 <Blob: images.trumpspired.com, generated/031ceaef62494d7bbabcd7614573b968.jpg, 1567649912447589>,
 <Blob: images.trumpspired.com, generated/0378e9b28dc946989196663c919ff4db.jpg, 1568781548361217>,
 <Blob: images.trumpspired.com, generated/037d4dfa0bd7471fbf9b50d060911921.jpg, 1578350393236392>,
 <Blob: images.trumpspired.com, generated/06a487cd37104a37bfd4146a4d1f43ce.jpg, 1567609268692580>,
 <Blob: images.trumpspired.com, generated/0759

In [158]:
blob = bucket.get_blob('generated/002fd842e6f74ddebdd88895bdf2b499.jpg')

In [4]:
from lib.tweemio import filepersist

In [16]:
importlib.reload(filepersist)

<module 'lib.tweemio.filepersist' from '/Users/liangjh/Workspace/tweemio-api/lib/tweemio/filepersist.py'>

In [5]:
client = filepersist.gcs_client()

In [7]:
bucket = client.get_bucket('images.trumpspired.com')

In [39]:
import uuid

In [40]:
guid = uuid.uuid4().hex

In [43]:
mdl = asmbl_models.TwmUserCalc.create(
        screen_name='jliang1', grp='trumpian', uuid=guid)

In [44]:
mdl.update()

<TwmUserCalc>

In [46]:
mdl.delete()

<TwmUserCalc>

In [48]:
datetime.date.today().strftime('%Y%m%d')

'20200201'